In [1]:
# FX Risk Analytics & Currency Hedging
# Valuuttariskin analyysi ja suojausstrategiat

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')


In [2]:

class FXRiskAnalyzer:
    def __init__(self):
        self.fx_data = None
        self.returns_data = None
        self.volatility_models = {}
        self.correlation_matrix = None
        self.hedging_strategies = {}
        
    def fetch_fx_data(self, currency_pairs, start_date='2020-01-01', end_date='2024-01-01'):
        """Haetaan valuuttakurssidataa"""
        print("💱 Haetaan valuuttakurssidataa...")
        
        # Muunnetaan currency_pairit Yahoo Finance -muotoon
        yahoo_symbols = []
        self.currency_pairs = currency_pairs
        
        for pair in currency_pairs:
            if pair == 'EURUSD':
                yahoo_symbols.append('EURUSD=X')
            elif pair == 'GBPUSD':
                yahoo_symbols.append('GBPUSD=X')
            elif pair == 'USDJPY':
                yahoo_symbols.append('USDJPY=X')
            elif pair == 'USDCHF':
                yahoo_symbols.append('USDCHF=X')
            elif pair == 'AUDUSD':
                yahoo_symbols.append('AUDUSD=X')
            elif pair == 'USDCAD':
                yahoo_symbols.append('USDCAD=X')
            elif pair == 'NZDUSD':
                yahoo_symbols.append('NZDUSD=X')
            elif pair == 'EURGBP':
                yahoo_symbols.append('EURGBP=X')
            elif pair == 'EURJPY':
                yahoo_symbols.append('EURJPY=X')
            elif pair == 'GBPJPY':
                yahoo_symbols.append('GBPJPY=X')
        
        # Haetaan data
        fx_data = {}
        for i, symbol in enumerate(yahoo_symbols):
            try:
                data = yf.download(symbol, start=start_date, end=end_date, progress=False)
                if not data.empty:
                    # Käsittele MultiIndex
                    if hasattr(data.columns, 'levels'):
                        close_price = data['Close'].iloc[:, 0] if len(data['Close'].shape) > 1 else data['Close']
                    else:
                        close_price = data['Close']
                    
                    fx_data[currency_pairs[i]] = close_price
                    print(f"✓ {currency_pairs[i]}: {len(close_price)} päivää")
                else:
                    print(f"✗ {currency_pairs[i]}: Ei dataa")
            except Exception as e:
                print(f"✗ {currency_pairs[i]}: {e}")
        
        self.fx_data = pd.DataFrame(fx_data).dropna()
        self.returns_data = self.fx_data.pct_change().dropna()
        
        print(f"\n📊 FX-data ladattu:")
        print(f"   Valuuttaparit: {len(self.fx_data.columns)}")
        print(f"   Aikajakso: {len(self.fx_data)} päivää")
        
        return self.fx_data
    
    def fx_market_analysis(self):
        """Analysoidaan FX-markkinoiden ominaisuuksia"""
        print("\n💱 FX-MARKKINOIDEN ANALYYSI:")
        print("-" * 50)
        
        # Volatiliteetit
        annualized_vols = self.returns_data.std() * np.sqrt(252)
        
        print("📊 VUOSIVOLATILITEETIT:")
        for pair, vol in annualized_vols.items():
            print(f"{pair:8}: {vol:.2%}")
        
        # Korrelaatiot
        self.correlation_matrix = self.returns_data.corr()
        
        print(f"\n🔗 KORRELAATIOT:")
        print("Korkein positiivinen korrelaatio:")
        # Etsitään korkein korrelaatio (pois lukien diagonaali)
        corr_values = self.correlation_matrix.values
        np.fill_diagonal(corr_values, 0)  # Nollaa diagonaali
        max_corr_idx = np.unravel_index(np.argmax(corr_values), corr_values.shape)
        max_corr_pair = (self.correlation_matrix.index[max_corr_idx[0]], 
                        self.correlation_matrix.columns[max_corr_idx[1]])
        max_corr_value = corr_values[max_corr_idx]
        
        print(f"{max_corr_pair[0]} - {max_corr_pair[1]}: {max_corr_value:.3f}")
        
        # Skewness ja Kurtosis (FX-markkinoiden erityispiirteet)
        print(f"\n📈 JAKAUMIEN OMINAISUUDET:")
        for pair in self.currency_pairs:
            if pair in self.returns_data.columns:
                returns = self.returns_data[pair]
                skew = stats.skew(returns)
                kurt = stats.kurtosis(returns)
                
                # Jarque-Bera -testi normaalisuudelle
                jb_stat, jb_p = stats.jarque_bera(returns)
                
                print(f"{pair:8}: Skew={skew:6.3f}, Kurt={kurt:6.3f}, JB p-val={jb_p:.4f}")
        
        # Carry Trade -analyysi (simuloitu korkoerot)
        print(f"\n💰 CARRY TRADE POTENTIAALI (simuloitu):")
        # Simuloidaan korkoerot (todellisuudessa haettaisiin keskuspankeista)
        simulated_rates = {
            'EURUSD': {'EUR': 0.005, 'USD': 0.045},  # Fed vs ECB
            'GBPUSD': {'GBP': 0.025, 'USD': 0.045},  # BoE vs Fed
            'USDJPY': {'USD': 0.045, 'JPY': -0.001}, # Fed vs BoJ
            'USDCHF': {'USD': 0.045, 'CHF': 0.001},  # Fed vs SNB
            'AUDUSD': {'AUD': 0.035, 'USD': 0.045},  # RBA vs Fed
        }
        
        for pair in simulated_rates:
            if pair in self.returns_data.columns:
                base_curr = pair[:3]
                quote_curr = pair[3:]
                rate_diff = simulated_rates[pair][base_curr] - simulated_rates[pair][quote_curr]
                
                # Carry trade tuotto = korkoerot + valuuttakurssimuutos
                fx_return = self.returns_data[pair].mean() * 252
                carry_return = rate_diff + fx_return
                
                print(f"{pair}: Korkoerotus={rate_diff:+.3%}, FX-tuotto={fx_return:+.3%}, Total={carry_return:+.3%}")
        
        return {
            'volatilities': annualized_vols,
            'correlations': self.correlation_matrix,
            'max_correlation': (max_corr_pair, max_corr_value)
        }
    
    def purchasing_power_parity_analysis(self):
        """Analysoidaan ostovoimapariteettia (PPP)"""
        print("\n🏛️ OSTOVOIMAPARITEETTI (PPP) ANALYYSI:")
        print("-" * 50)
        
        # Big Mac Index -simulaatio (yksinkertainen PPP-mittari)
        print("🍔 BIG MAC INDEX (simuloitu):")
        
        # Simuloidaan Big Mac -hintoja eri maissa (USD)
        big_mac_prices = {
            'USD': 5.50,   # USA baseline
            'EUR': 4.20,   # Eurooppa
            'GBP': 3.80,   # UK
            'JPY': 390,    # Japani (jeninä)
            'CHF': 6.20,   # Sveitsi
            'AUD': 6.80,   # Australia
            'CAD': 6.50,   # Kanada
        }
        
        # Lasketaan implisiittiset valuuttakurssit
        usd_price = big_mac_prices['USD']
        
        for currency, price in big_mac_prices.items():
            if currency != 'USD':
                if currency == 'JPY':
                    implied_rate = price / usd_price  # JPY per USD
                    pair_name = 'USDJPY'
                elif currency == 'EUR':
                    implied_rate = usd_price / price  # USD per EUR (sitten käännetään)
                    implied_rate = 1 / implied_rate   # EUR per USD
                    pair_name = 'EURUSD'
                elif currency == 'GBP':
                    implied_rate = usd_price / price
                    implied_rate = 1 / implied_rate
                    pair_name = 'GBPUSD'
                elif currency == 'CHF':
                    implied_rate = price / usd_price
                    pair_name = 'USDCHF'
                elif currency == 'AUD':
                    implied_rate = usd_price / price
                    implied_rate = 1 / implied_rate
                    pair_name = 'AUDUSD'
                elif currency == 'CAD':
                    implied_rate = price / usd_price
                    pair_name = 'USDCAD'
                
                # Verrataan nykyiseen kurssiin
                if pair_name in self.fx_data.columns:
                    current_rate = self.fx_data[pair_name].iloc[-1]
                    over_under_valued = (implied_rate - current_rate) / current_rate * 100
                    
                    print(f"{pair_name}: PPP={implied_rate:.4f}, Nykyinen={current_rate:.4f}, "
                          f"{'Yliarvostettu' if over_under_valued > 0 else 'Aliarvostettu'}={abs(over_under_valued):.1f}%")
    
    def volatility_modeling_garch(self):
        """GARCH-volatiliteettimallintaminen FX-pareille"""
        print("\n📊 GARCH VOLATILITEETTIMALLINTAMINEN:")
        print("-" * 50)
        
        # Yksinkertainen GARCH(1,1) simulaatio
        # Todellisuudessa käytettäisiin arch-kirjastoa
        
        garch_results = {}
        
        for pair in self.currency_pairs:
            if pair in self.returns_data.columns:
                returns = self.returns_data[pair].dropna()
                
                # Estimoidaan GARCH(1,1) parametrit (yksinkertainen versio)
                # σ²(t) = ω + α*ε²(t-1) + β*σ²(t-1)
                
                # Käytetään rolling window volatiliteettia GARCH-approksimointina
                window = 30
                rolling_vol = returns.rolling(window).std()
                
                # Yksinkertainen GARCH-simulaatio
                alpha = 0.1  # ARCH-kerroin
                beta = 0.85  # GARCH-kerroin  
                omega = rolling_vol.var().mean() * (1 - alpha - beta)
                
                # Ennustetaan seuraavan päivän volatiliteetti
                last_return_sq = returns.iloc[-1] ** 2
                last_vol_sq = rolling_vol.iloc[-1] ** 2
                
                next_vol_sq = omega + alpha * last_return_sq + beta * last_vol_sq
                next_vol = np.sqrt(next_vol_sq)
                
                garch_results[pair] = {
                    'omega': omega,
                    'alpha': alpha,
                    'beta': beta,
                    'current_vol': rolling_vol.iloc[-1],
                    'predicted_vol': next_vol,
                    'annualized_vol': next_vol * np.sqrt(252)
                }
                
                print(f"{pair}: Current Vol={rolling_vol.iloc[-1]:.4f}, "
                      f"Predicted={next_vol:.4f}, "
                      f"Annualized={next_vol * np.sqrt(252):.2%}")
        
        self.volatility_models = garch_results
        return garch_results
    
    def currency_exposure_analysis(self, portfolio_exposures):
        """Analysoi portfolion valuuttariskit"""
        print("\n💼 PORTFOLIO VALUUTTARISKI-ANALYYSI:")
        print("-" * 50)
        
        # portfolio_exposures = {'USD': 0.4, 'EUR': 0.3, 'GBP': 0.2, 'JPY': 0.1}
        
        print("🌍 PORTFOLIO VALUUTTAJAKAUMA:")
        for currency, exposure in portfolio_exposures.items():
            print(f"{currency}: {exposure:.1%}")
        
        # Lasketaan portfolio-volatiliteetti valuuttariskistä
        # Tarvitaan cross-rates eurolle (base currency)
        
        # Simuloidaan portfolio FX-riski
        base_currency = 'EUR'  # Oletetaan euro base currency
        
        fx_portfolio_returns = []
        
        for date in self.returns_data.index:
            daily_fx_return = 0
            
            for currency, weight in portfolio_exposures.items():
                if currency == base_currency:
                    continue
                    
                # Etsitään sopiva FX-pari
                fx_return = 0
                if currency == 'USD' and 'EURUSD' in self.returns_data.columns:
                    fx_return = -self.returns_data.loc[date, 'EURUSD']  # USD heikkenee = neg tuotto EUR-sijoittajalle
                elif currency == 'GBP' and 'EURGBP' in self.returns_data.columns:
                    fx_return = -self.returns_data.loc[date, 'EURGBP']
                elif currency == 'JPY' and 'EURJPY' in self.returns_data.columns:
                    fx_return = -self.returns_data.loc[date, 'EURJPY']
                
                daily_fx_return += weight * fx_return
            
            fx_portfolio_returns.append(daily_fx_return)
        
        fx_portfolio_returns = pd.Series(fx_portfolio_returns, index=self.returns_data.index)
        
        # FX-riskin mittarit
        fx_vol = fx_portfolio_returns.std() * np.sqrt(252)
        fx_var_95 = np.percentile(fx_portfolio_returns, 5)
        fx_var_99 = np.percentile(fx_portfolio_returns, 1)
        
        print(f"\n📊 PORTFOLIO FX-RISKIN MITTARIT:")
        print(f"Vuosivolatiliteetti: {fx_vol:.2%}")
        print(f"Päivittäinen VaR 95%: {fx_var_95:.2%}")
        print(f"Päivittäinen VaR 99%: {fx_var_99:.2%}")
        
        # 1M€ portfoliolle
        portfolio_value = 1000000
        print(f"\n💰 1M€ PORTFOLIOLLE:")
        print(f"VaR 95%: {abs(fx_var_95) * portfolio_value:,.0f}€")
        print(f"VaR 99%: {abs(fx_var_99) * portfolio_value:,.0f}€")
        
        return {
            'fx_returns': fx_portfolio_returns,
            'fx_volatility': fx_vol,
            'var_95': fx_var_95,
            'var_99': fx_var_99
        }
    
    def hedging_strategies_analysis(self, portfolio_exposures, hedge_ratios=[0.0, 0.25, 0.5, 0.75, 1.0]):
        """Analysoi eri suojausstrategioita"""
        print("\n🛡️ VALUUTTASUOJAUSSTRATEGIAT:")
        print("-" * 50)
        
        # Simuloidaan eri hedge ratiot
        base_currency = 'EUR'
        
        hedging_results = {}
        
        for hedge_ratio in hedge_ratios:
            print(f"\n📈 HEDGE RATIO: {hedge_ratio:.0%}")
            
            # Lasketaan suojattu portfolio
            hedged_returns = []
            hedging_costs = []
            
            for date in self.returns_data.index:
                daily_return = 0
                daily_hedge_cost = 0
                
                for currency, weight in portfolio_exposures.items():
                    if currency == base_currency:
                        continue
                    
                    # Valuuttariski ilman suojausta
                    fx_return = 0
                    if currency == 'USD' and 'EURUSD' in self.returns_data.columns:
                        fx_return = -self.returns_data.loc[date, 'EURUSD']
                    elif currency == 'GBP' and 'EURGBP' in self.returns_data.columns:
                        fx_return = -self.returns_data.loc[date, 'EURGBP']
                    
                    # Suojattu osuus
                    unhedged_return = fx_return * (1 - hedge_ratio)
                    
                    # Suojauskustannukset (bid-ask spread + forward premium)
                    if hedge_ratio > 0:
                        hedge_cost = 0.0001 * hedge_ratio * weight  # 1bp per päivä suojauksesta
                        daily_hedge_cost += hedge_cost
                    
                    daily_return += weight * unhedged_return
                
                hedged_returns.append(daily_return)
                hedging_costs.append(daily_hedge_cost)
            
            hedged_returns = pd.Series(hedged_returns, index=self.returns_data.index)
            total_hedging_cost = sum(hedging_costs)
            
            # Suojauksen tehokkuuden mittarit
            hedged_vol = hedged_returns.std() * np.sqrt(252)
            hedged_var_95 = np.percentile(hedged_returns, 5)
            
            print(f"   Volatiliteetti: {hedged_vol:.2%}")
            print(f"   VaR 95%: {hedged_var_95:.2%}")
            print(f"   Vuotuiset suojauskustannukset: {total_hedging_cost:.4%}")
            
            hedging_results[hedge_ratio] = {
                'volatility': hedged_vol,
                'var_95': hedged_var_95,
                'hedging_cost': total_hedging_cost,
                'returns': hedged_returns
            }
        
        # Optimaalinen hedge ratio
        optimal_ratio = min(hedging_results.keys(), 
                           key=lambda x: hedging_results[x]['volatility'])
        
        print(f"\n🎯 OPTIMAALINEN HEDGE RATIO: {optimal_ratio:.0%}")
        print(f"   (Minimoi volatiliteetin)")
        
        self.hedging_strategies = hedging_results
        return hedging_results
    
    def fx_forecasting_model(self):
        """Yksinkertainen FX-ennustemalli"""
        print("\n🔮 FX-ENNUSTEMALLI:")
        print("-" * 50)
        
        # Käytetään EURUSD:ta esimerkkinä
        if 'EURUSD' not in self.fx_data.columns:
            print("❌ EURUSD-data puuttuu ennustemallia varten")
            return None
        
        # Feature engineering
        eurusd_data = self.fx_data['EURUSD'].copy()
        
        # Tekniset indikaattorit
        def calculate_technical_indicators(prices):
            data = pd.DataFrame({'price': prices})
            
            # Moving averages
            data['sma_10'] = data['price'].rolling(10).mean()
            data['sma_50'] = data['price'].rolling(50).mean()
            data['sma_200'] = data['price'].rolling(200).mean()
            
            # RSI
            delta = data['price'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            data['rsi'] = 100 - (100 / (1 + rs))
            
            # MACD
            ema_12 = data['price'].ewm(span=12).mean()
            ema_26 = data['price'].ewm(span=26).mean()
            data['macd'] = ema_12 - ema_26
            data['macd_signal'] = data['macd'].ewm(span=9).mean()
            
            # Bollinger Bands
            data['bb_middle'] = data['price'].rolling(20).mean()
            bb_std = data['price'].rolling(20).std()
            data['bb_upper'] = data['bb_middle'] + (bb_std * 2)
            data['bb_lower'] = data['bb_middle'] - (bb_std * 2)
            data['bb_position'] = (data['price'] - data['bb_lower']) / (data['bb_upper'] - data['bb_lower'])
            
            # Volatiliteetti
            data['volatility'] = data['price'].pct_change().rolling(20).std()
            
            return data
        
        technical_data = calculate_technical_indicators(eurusd_data)
        
        # Luodaan features ja target
        features = [
            'sma_10', 'sma_50', 'sma_200', 'rsi', 'macd', 'macd_signal',
            'bb_position', 'volatility'
        ]
        
        # Target: seuraavan päivän tuotto
        technical_data['target'] = technical_data['price'].pct_change().shift(-1)
        
        # Normalisoidaan features
        for feature in features:
            technical_data[f'{feature}_norm'] = (technical_data[feature] - technical_data[feature].mean()) / technical_data[feature].std()
        
        feature_cols = [f'{f}_norm' for f in features]
        
        # Poistetaan puuttuvat arvot
        model_data = technical_data[feature_cols + ['target']].dropna()
        
        if len(model_data) < 100:
            print("❌ Liian vähän dataa mallintamiseen")
            return None
        
        # Jaetaan data
        X = model_data[feature_cols]
        y = model_data['target']
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=False  # Älä sekoita aikasarjaa
        )
        
        # Koulutetaan Random Forest -malli
        model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
        model.fit(X_train, y_train)
        
        # Ennusteet
        y_pred = model.predict(X_test)
        
        # Mittarit
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        print(f"📊 MALLIN SUORITUSKYKY:")
        print(f"RMSE: {rmse:.6f}")
        print(f"R²: {r2:.3f}")
        
        # Feature importance
        feature_importance = pd.DataFrame({
            'feature': features,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\n🔍 TÄRKEIMMÄT ENNUSTETEKIJÄT:")
        for _, row in feature_importance.head(5).iterrows():
            print(f"{row['feature']:15}: {row['importance']:.3f}")
        
        # Kaupankäyntisignaalit
        print(f"\n📈 KAUPANKÄYNTISIGNAALIT:")
        
        # Luokitellaan ennusteet
        strong_buy_threshold = np.percentile(y_pred, 95)
        buy_threshold = np.percentile(y_pred, 80)
        sell_threshold = np.percentile(y_pred, 20)
        strong_sell_threshold = np.percentile(y_pred, 5)
        
        signals = []
        for pred in y_pred[-10:]:  # Viimeiset 10 ennustetta
            if pred > strong_buy_threshold:
                signals.append("🟢 STRONG BUY")
            elif pred > buy_threshold:
                signals.append("🔵 BUY")
            elif pred < strong_sell_threshold:
                signals.append("🔴 STRONG SELL")
            elif pred < sell_threshold:
                signals.append("🟠 SELL")
            else:
                signals.append("⚪ HOLD")
        
        for i, signal in enumerate(signals[-5:]):  # Näytä viimeiset 5
            print(f"Päivä -{5-i}: {signal}")
        
        return {
            'model': model,
            'rmse': rmse,
            'r2': r2,
            'feature_importance': feature_importance,
            'predictions': y_pred,
            'actuals': y_test
        }
    
    def create_fx_visualizations(self):
        """Luodaan kattavat FX-visualisoinnit"""
        print("\n📊 Luodaan FX-visualisoinnit...")
        
        fig, axes = plt.subplots(3, 2, figsize=(16, 18))
        
        # 1. Valuuttakurssien kehitys
        normalized_fx = self.fx_data / self.fx_data.iloc[0]
        normalized_fx.plot(ax=axes[0,0], title='Valuuttakurssien kehitys (normalisoitu)', 
                          legend=True)
        axes[0,0].set_ylabel('Normalisoitu arvo')
        axes[0,0].grid(True, alpha=0.3)
        
        # 2. Volatiliteettien vertailu
        rolling_vols = self.returns_data.rolling(30).std() * np.sqrt(252)
        rolling_vols.plot(ax=axes[0,1], title='30pv rullaava volatiliteetti (vuositasolla)')
        axes[0,1].set_ylabel('Volatiliteetti')
        axes[0,1].grid(True, alpha=0.3)
        
        # 3. Tuottojen jakaumat
        for i, pair in enumerate(self.currency_pairs[:4]):  # Näytä max 4
            if pair in self.returns_data.columns:
                self.returns_data[pair].hist(bins=50, alpha=0.7, ax=axes[1,0], 
                                           label=pair, density=True)
        
        axes[1,0].set_title('Päivittäisten tuottojen jakaumat')
        axes[1,0].set_xlabel('Päivittäinen tuotto')
        axes[1,0].set_ylabel('Tiheys')
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
        
        # 4. Korrelaatiomatriisi
        if self.correlation_matrix is not None:
            sns.heatmap(self.correlation_matrix, annot=True, cmap='RdYlBu_r', center=0,
                       square=True, ax=axes[1,1], fmt='.2f')
            axes[1,1].set_title('Valuuttaparien korrelaatiomatriisi')
        
        # 5. VaR-analyysi
        var_data = []
        for pair in self.currency_pairs:
            if pair in self.returns_data.columns:
                returns = self.returns_data[pair]
                var_95 = np.percentile(returns, 5)
                var_99 = np.percentile(returns, 1)
                var_data.append({'Pair': pair, 'VaR_95': abs(var_95), 'VaR_99': abs(var_99)})
        
        if var_data:
            var_df = pd.DataFrame(var_data)
            x_pos = np.arange(len(var_df))
            width = 0.35
            
            axes[2,0].bar(x_pos - width/2, var_df['VaR_95'], width, 
                         label='VaR 95%', alpha=0.8, color='orange')
            axes[2,0].bar(x_pos + width/2, var_df['VaR_99'], width,
                         label='VaR 99%', alpha=0.8, color='red')
            
            axes[2,0].set_title('Value at Risk -vertailu')
            axes[2,0].set_xlabel('Valuuttapari')
            axes[2,0].set_ylabel('VaR (abs)')
            axes[2,0].set_xticks(x_pos)
            axes[2,0].set_xticklabels(var_df['Pair'], rotation=45)
            axes[2,0].legend()
            axes[2,0].grid(True, alpha=0.3)
        
        

In [3]:
# Nopea testi pienemmällä datalla
fx_analyzer = FXRiskAnalyzer()
test_pairs = ['EURUSD', 'GBPUSD']  # Vain 2 paria testiin
fx_analyzer.fetch_fx_data(test_pairs, start_date='2023-01-01')
fx_analyzer.fx_market_analysis()

💱 Haetaan valuuttakurssidataa...
✓ EURUSD: 260 päivää
✓ GBPUSD: 260 päivää

📊 FX-data ladattu:
   Valuuttaparit: 2
   Aikajakso: 260 päivää

💱 FX-MARKKINOIDEN ANALYYSI:
--------------------------------------------------
📊 VUOSIVOLATILITEETIT:
EURUSD  : 7.53%
GBPUSD  : 8.20%

🔗 KORRELAATIOT:
Korkein positiivinen korrelaatio:
EURUSD - GBPUSD: 0.813

📈 JAKAUMIEN OMINAISUUDET:
EURUSD  : Skew= 0.118, Kurt= 0.514, JB p-val=0.1779
GBPUSD  : Skew= 0.025, Kurt= 0.801, JB p-val=0.0310

💰 CARRY TRADE POTENTIAALI (simuloitu):
EURUSD: Korkoerotus=-4.000%, FX-tuotto=+3.486%, Total=-0.514%
GBPUSD: Korkoerotus=-2.000%, FX-tuotto=+5.141%, Total=+3.141%


{'volatilities': EURUSD    0.075348
 GBPUSD    0.081982
 dtype: float64,
 'correlations':           EURUSD    GBPUSD
 EURUSD  0.000000  0.812878
 GBPUSD  0.812878  0.000000,
 'max_correlation': (('EURUSD', 'GBPUSD'), np.float64(0.8128783251565527))}

In [4]:
# Luodaan analyysiobjekti
fx_analyzer = FXRiskAnalyzer()

# Ajetaan vaihe kerrallaan
major_pairs = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCHF']
fx_analyzer.fetch_fx_data(major_pairs)
fx_analyzer.fx_market_analysis()

💱 Haetaan valuuttakurssidataa...
✓ EURUSD: 1043 päivää
✓ GBPUSD: 1043 päivää
✓ USDJPY: 1043 päivää
✓ USDCHF: 1043 päivää

📊 FX-data ladattu:
   Valuuttaparit: 4
   Aikajakso: 1043 päivää

💱 FX-MARKKINOIDEN ANALYYSI:
--------------------------------------------------
📊 VUOSIVOLATILITEETIT:
EURUSD  : 7.73%
GBPUSD  : 9.73%
USDJPY  : 9.06%
USDCHF  : 7.76%

🔗 KORRELAATIOT:
Korkein positiivinen korrelaatio:
EURUSD - GBPUSD: 0.704

📈 JAKAUMIEN OMINAISUUDET:
EURUSD  : Skew=-0.076, Kurt= 1.692, JB p-val=0.0000
GBPUSD  : Skew=-0.313, Kurt= 5.365, JB p-val=0.0000
USDJPY  : Skew=-0.303, Kurt= 4.443, JB p-val=0.0000
USDCHF  : Skew=-0.322, Kurt= 2.490, JB p-val=0.0000

💰 CARRY TRADE POTENTIAALI (simuloitu):
EURUSD: Korkoerotus=-4.000%, FX-tuotto=-0.033%, Total=-4.033%
GBPUSD: Korkoerotus=-2.000%, FX-tuotto=-0.510%, Total=-2.510%
USDJPY: Korkoerotus=+4.600%, FX-tuotto=+6.781%, Total=+11.381%
USDCHF: Korkoerotus=+4.400%, FX-tuotto=-3.012%, Total=+1.388%


{'volatilities': EURUSD    0.077298
 GBPUSD    0.097267
 USDJPY    0.090613
 USDCHF    0.077643
 dtype: float64,
 'correlations':           EURUSD    GBPUSD    USDJPY    USDCHF
 EURUSD  0.000000  0.703578 -0.451207 -0.774097
 GBPUSD  0.703578  0.000000 -0.423510 -0.581931
 USDJPY -0.451207 -0.423510  0.000000  0.527631
 USDCHF -0.774097 -0.581931  0.527631  0.000000,
 'max_correlation': (('EURUSD', 'GBPUSD'), np.float64(0.703577617696478))}